In [1]:
from cxxwrap.pycxx import py11

from cxxwrap.tools import set_args
from cxxwrap.types import *

pybind11 headers path /home/matrix/.local/lib/python3.10/site-packages/pybind11/include
python headers path /usr/include/python3.10


In [2]:
a = set_args(flags = """-std=c++17 -I/home/matrix/GSoc/ipyhpx 
          -I/home/matrix/pyhpx/release_hpx/hpx/cmake_install/include 
          -L/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib -lhpx 
          -Wl,-rpath=/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib""",
          lib_path="./matrix",
          module_name="operations")

In [3]:
create_type("func","std::function<void()>")

In [4]:
@py11(headers=["<run_hpx.cpp>","<iostream>"],recompile=True,args=a)
def hpx_wrapper(f : func)->None:
    """
    const char *num = getenv("HPX_NUM_THREADS");
    int num_threads = num == 0 ? 4 : atoi(num);
    std::cout << "Using " << num_threads << " threads." << std::endl;
    hpx_global::submit_work(num_threads,f);
    """

self.flags =  -std=c++17 -I/home/matrix/GSoc/ipyhpx 
          -I/home/matrix/pyhpx/release_hpx/hpx/cmake_install/include 
          -L/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib -lhpx 
          -Wl,-rpath=/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib
self.module_name =  operations
self.lib_path =  ./matrix
input is ./matrix
absolute path 
Creating directory: /home/matrix/GSoc/cxxwrap/Examples/matrix
function name is : hpx_wrapper
input is ./matrix
absolute path 
Directory already exists: /home/matrix/GSoc/cxxwrap/Examples/matrix
the path is : /home/matrix/GSoc/cxxwrap/Examples/matrix/hpx_wrapper
type translation std::function<void()>
type translation void
started executing compilation ....
function  is : 
def hpx_wrapper(*args):
    fun = importlib.import_module("hpx_wrapper_v1","./matrix")
    fun.load()
    return fun.call(*args)
    
finished compilation 8.91 sec
The path where .so goes ./matrix


In [5]:
create_type("vvec","std::vector<std::vector<int>>",is_template=True)

In [6]:
@py11(args=a,
      headers=["<iostream>",
            "<vector>",
            "<hpx/algorithm.hpp>",
            "<hpx/chrono.hpp>",
            "<hpx/init.hpp>",
            "<hpx/modules/assertion.hpp>"],
        wrap=hpx_wrapper)
def matmul_par(a:vvec,b:vvec)->vvec:

      """
    using namespace std;
    int ra = a.size();
    int ca = a[0].size();
    int rb = b.size();
    int cb = b[0].size();

    if (ca != rb){
        cerr << "Dimensions incorrect (it's A(m,n)*B(n,k))"<< endl;
        return {};
    }

    vector<vector<int>> c(ra, vector<int>(cb,0));
    //
    hpx::for_each(hpx::execution::par, a.begin(), a.end(), [&](const std::vector<int>& row_a) {
        int i = &row_a - &a[0];
        for (int j = 0; j < cb; j++) {
            for (int k = 0; k < ca; k++) {
                c[i][j] += row_a[k] * b[k][j];
            }
        }
    });
    
    return c;


      """
    

self.flags =  -std=c++17 -I/home/matrix/GSoc/ipyhpx 
          -I/home/matrix/pyhpx/release_hpx/hpx/cmake_install/include 
          -L/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib -lhpx 
          -Wl,-rpath=/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib
self.module_name =  operations
self.lib_path =  ./matrix
input is ./matrix
absolute path 
Directory already exists: /home/matrix/GSoc/cxxwrap/Examples/matrix
function name is : matmul_par
input is ./matrix
absolute path 
Directory already exists: /home/matrix/GSoc/cxxwrap/Examples/matrix
the path is : /home/matrix/GSoc/cxxwrap/Examples/matrix/matmul_par
type translation std::vector<std::vector<int>>
type translation std::vector<std::vector<int>>
type translation std::vector<std::vector<int>>
started executing compilation ....
function  is : 
def matmul_par(*args):
    fun = importlib.import_module("matmul_par_v1","./matrix")
    fun.load()
    return fun.call(*args)
    
function was updated
finished compilation 9.01 sec


```cpp
vector<vector<int>> matmul_par( const vector<vector<int>>& a,
                            const vector<vector<int>>& b, 
                            bool flag = 1) {
    

    auto start = std::chrono::high_resolution_clock::now();
    int ra = a.size();
    int ca = a[0].size();
    int rb = b.size();
    int cb = b[0].size();

    if (ca != rb){
        cerr << "Dimensions incorrect (it's A(m,n)*B(n,k))"<< endl;
        return {};
    }

    vector<vector<int>> c(ra, vector<int>(cb,0));
    //
    hpx::for_each(hpx::execution::par, a.begin(), a.end(), [&](const std::vector<int>& row_a) {
        int i = &row_a - &a[0];
        for (int j = 0; j < cb; j++) {
            for (int k = 0; k < ca; k++) {
                c[i][j] += row_a[k] * b[k][j];
            }
        }
    });
    
    //printing the time taken 
    auto end = std::chrono::high_resolution_clock::now();
    auto duration = std::chrono::duration_cast<std::chrono::milliseconds>(end - start);
    if (flag == 1){
        std::cout   << "Time taken by  parallel(par) version: " 
                    << duration.count() 
                    << "ms" << endl; 
    }
    return c;
}
```

In [7]:
@py11(args=a,headers=["<iostream>","<vector>", "<random>"],recompile=True)

def generateMatrix(rows:int, cols:int)->vvec: 
    """
    using namespace std;
    // Create a random number generator
    random_device rd;
    mt19937 gen(rd());
    uniform_int_distribution<int> dist(1, 100);

    // Create the matrix and fill it with random values
    vector<vector<int>> matrix(rows, vector<int>(cols));
    for (int i = 0; i < rows; ++i) {
        for (int j = 0; j < cols; ++j) {
            matrix[i][j] = dist(gen);
        }
    }
    std::cout<<"hello";

    return matrix;
    """


self.flags =  -std=c++17 -I/home/matrix/GSoc/ipyhpx 
          -I/home/matrix/pyhpx/release_hpx/hpx/cmake_install/include 
          -L/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib -lhpx 
          -Wl,-rpath=/home/matrix/pyhpx/release_hpx/hpx/cmake_install/lib
self.module_name =  operations
self.lib_path =  ./matrix
input is ./matrix
absolute path 
Directory already exists: /home/matrix/GSoc/cxxwrap/Examples/matrix
function name is : generateMatrix
input is ./matrix
absolute path 
Directory already exists: /home/matrix/GSoc/cxxwrap/Examples/matrix
the path is : /home/matrix/GSoc/cxxwrap/Examples/matrix/generateMatrix
type translation int
type translation int
type translation std::vector<std::vector<int>>
started executing compilation ....
function  is : 
def generateMatrix(*args):
    fun = importlib.import_module("generateMatrix_v1","./matrix")
    fun.load()
    return fun.call(*args)
    
function was updated
finished compilation 3.15 sec
The path where .so goes ./matrix


In [8]:
a = generateMatrix(2,2)
b = generateMatrix(2,2)

hellohello

In [9]:
print(a)
print(b)

[[7, 70], [79, 96]]
[[82, 93], [56, 37]]


In [10]:
c =matmul_par(a,b)

Using 4 threads.


In [11]:
c

[[4494, 3241], [11854, 10899]]

In [12]:
from matrix.operations import matmul_par as multiply